In [ ]:
# Mount gdrive

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# Change Directory

In [ ]:
%cd "/content/drive/My Drive/UCM/tfm_code/notebooks"

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-25-e8e68b7a788d>", line 1, in <module>
    get_ipython().run_line_magic('cd', '"/content/drive/My Drive/UCM/tfm_code/notebooks"')
  File "/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py", line 2314, in run_line_magic
    result = fn(*args, **kwargs)
  File "<decorator-gen-85>", line 2, in cd
  File "/usr/local/lib/python3.9/dist-packages/IPython/core/magic.py", line 187, in <lambda>
    call = lambda f, *a, **k: f(*a, **k)
  File "/usr/local/lib/python3.9/dist-packages/IPython/core/magics/osm.py", line 334, in cd
    oldcwd = os.getcwd()
OSError: [Errno 107] Transport endpoint is not connected

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/IPytho

OSError: ignored

In [ ]:
# Import Libraries

In [ ]:
!pip install optuna

In [ ]:
# Downloaded
import optuna
import pandas as pd
import pyarrow
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.model_selection import  StratifiedKFold
import matplotlib.pyplot as plt

In [ ]:
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_param_importances

In [ ]:
from warnings import filterwarnings
filterwarnings('ignore')

In [ ]:
# Codigo

In [ ]:
df = pd.read_parquet(engine="pyarrow", path="/content/drive/My Drive/UCM/tfm_code/data/data_modelo_train_cleaned.parquet")

In [ ]:
df = df[df['SEMANA'].isin([17,16,15,14])]
df.drop(columns=['CLIENTE_ID','PRODUCTO_ID','ANIO','SEMANA'], inplace=True)

In [ ]:
important_columns =['FRECUENCIA', 'RECENCY', 'DURATION_CLIENT', 'TAMANIO',
       'UNIDAD_EMPAQUE', 'CANAL_RSR', 'SUBCANAL_OTHER',
       'SUBCANAL_VINATERÍA', 'MARCA_CIEL', 'MARCA_COCA-COLA',
       'MARCA_DEL VALLE', 'MARCA_FANTA', 'MARCA_FRESCA', 'MARCA_FUZE',
       'MARCA_MONSTER - PREDATOR', 'MARCA_MUNDET', 'MARCA_SANTA CLARA',
       'MARCA_SPRITE', 'MARCA_YOLI - SENZAO-AMEYAL', 'SABOR_COLA',
       'SABOR_FRUTAS', 'SABOR_LIMA-LIMON', 'SABOR_MANGO', 'SABOR_MORAS',
       'SABOR_NATURAL', 'SABOR_UVA', 'EMPAQUE_PET', 'EMPAQUE_REF PET',
       'EMPAQUE_VIDRIO', 'TIPO_CONSUMO_PERSONALES (SS)', 'CLUSTER_1',
       'CLUSTER_2', 'ROTATION_MEAN_CF', 'ROTATION_MEAN_DAYS', 'CF_LOG']

In [ ]:
X = df.drop(columns=['PURCHASE'], axis=1)
X = X[important_columns]
y = df['PURCHASE']

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, stratify=y,random_state=42)

In [ ]:
# del(df)
# del(X)
# del(y)
# del(X_train)
# del(y_train)

In [ ]:
np.random.seed(1234)

In [ ]:
# Split your data into features (X) and target variable (y)
X_train, X, y_train, y = train_test_split(X, y, test_size=0.14, stratify=y,random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y,random_state=42)

In [ ]:
X_train['FRECUENCIA'].count()

In [ ]:
# Define the objective function for Optuna
def objective(trial):
    params = {
        'objective': 'binary:logistic',
        #'eval_metric': 'aucpr',
        'booster': 'gbtree',
        'n_jobs': -1,
        'verbosity': 0,
        'tree_method': trial.suggest_categorical('tree_method',  ['hist','exact','approx']),
        'grow_policy': 'lossguide',
        'max_bin': trial.suggest_int('max_bin', 256, 512),
        'max_depth': trial.suggest_int('max_depth', 5, 16),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.005, 0.5),
        'subsample': trial.suggest_discrete_uniform('subsample', 0.5, 1, 0.1),
        'colsample_bytree': trial.suggest_discrete_uniform('colsample_bytree', 0.5, 1, 0.1),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 100.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 100.0),
        'gamma': trial.suggest_loguniform('gamma', 1e-8, 100.0),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'scale_pos_weight': trial.suggest_int('scale_pos_weight', 2, 10)
    }
    # Train and evaluate the model using the given hyperparameters
    clf = xgb.XGBClassifier(**params)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    f1 = f1_score(y_test, y_pred)
    return f1

# Run hyperparameter tuning with Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

# Print the best hyperparameters and the corresponding ROC AUC score
print('Best Average Precision Score:', study.best_value)
print('Best hyperparameters:', study.best_params)


In [ ]:
plot_optimization_history(study)

In [ ]:
plot_param_importances(study)